In [ ]:
using LilGuys
using CairoMakie
using Arya

In [ ]:
import TOML
using CSV, DataFrames

In [ ]:
using OrderedCollections

In [ ]:
function load_profile(galaxyname; ext="")
    prof = LilGuys.StellarProfile(joinpath(galaxyname, "density_profiles/jax$(ext)_profile.toml"))
  
    norm = LilGuys.lerp(prof.log_r, prof.log_Sigma)(0)
    prof.log_Sigma .-= norm
    return prof
end

In [ ]:
function load_props(galaxyname)
    props = TOML.parsefile(joinpath(galaxyname, "observed_properties.toml"))

    return props
end

In [ ]:
function r_ioo(profile::LilGuys.StellarProfile; q=0.1)
    if length(profile.quantiles) == 0
        return NaN
    end
    
    qs = OrderedDict(a => b for (a, b) in zip(profile.quantiles, profile.log_r_quantiles))
    return 10^qs[1-q] / 10^qs[q]
end

In [ ]:
function r_ioo(prof::LilGuys.SphericalProfile; q=0.1)
    Mt = LilGuys.calc_M_2D(prof, 100)
    q1 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - q, 1)
    q2 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - (1-q), 1)

    return q2/q1
end

In [ ]:
function r_oom(prof::LilGuys.SphericalProfile; q=0.1)
    Mt = LilGuys.calc_M_2D(prof, 100)
    q1 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - 0.5, 1)
    q2 = LilGuys.find_zero(x -> LilGuys.calc_M_2D(prof, x) / Mt - (1-q), 1)

    return q2/q1
end

In [ ]:
function r_oom(profile::LilGuys.StellarProfile; q=0.1)
    if length(profile.quantiles) == 0
        return NaN
    end
    
    qs = OrderedDict(a => b for (a, b) in zip(profile.quantiles, profile.log_r_quantiles))
    return 10^qs[1-q] / 10^qs[0.5]
end

In [ ]:
prof_ana = LilGuys.Exp3D(r_s = 1/2.026996389650345, M=1)

In [ ]:
r_quant_exp = OrderedDict()
for q in [0.01, 0.05, 0.1, 0.16, 0.25]
    r = r_ioo(prof_ana, q=q)
    println("$q\t$(r)")
    r_quant_exp[q] = r
end

In [ ]:
r_quant_exp_2d = OrderedDict()

for q in [0.01, 0.05, 0.1, 0.16, 0.25]
    r = r_ioo(LilGuys.Exp2D(), q=q)
    println("$q\t$(r)")
    r_quant_exp_2d[q] = r
end

In [ ]:
r_quant_logcusp = OrderedDict()

for q in [0.01, 0.05, 0.1, 0.16, 0.25]
    r = r_ioo(LilGuys.LogCusp2D(), q=q)
    println("$q\t$(r)")

    r_quant_logcusp[q] = r
end

In [ ]:
r_quant_plummer = OrderedDict()

for q in [0.01, 0.05, 0.1, 0.16, 0.25]
    r = r_ioo(LilGuys.Plummer(), q=q)
    println("$q\t$(r)")

    r_quant_plummer[q] = r
end

In [ ]:
r_quant_king = OrderedDict()

for q in [0.01, 0.05, 0.1, 0.16, 0.25]
    r = r_ioo(LilGuys.KingProfile(R_t=5, k=1), q=q)
    println("$q\t$(r)")

    r_quant_king[q] = r
end

In [ ]:
r_quant_king2 = OrderedDict()

for q in [0.01, 0.05, 0.1, 0.16, 0.25]
    r = r_ioo(LilGuys.KingProfile(R_t=2, k=1), q=q)
    println("$q\t$(r)")

    r_quant_king2[q] = r
end

In [ ]:
r_quants = OrderedDict(
    "exp3d" => r_quant_exp,
    "exp2d" => r_quant_exp_2d,
    "logcusp" => r_quant_logcusp,
    "plummer" => r_quant_plummer,
    "king" => r_quant_king,
)    

In [ ]:
profs_ana = OrderedDict(
    "exp3d" => prof,
    "exp2d" => LilGuys.Exp2D(),
    "logcusp" => LilGuys.LogCusp2D(),
    "plummer" => LilGuys.Plummer(),
    "king" => LilGuys.KingProfile(k=1, c=5),
    )

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "quantile",
    ylabel = L"\log\,r_{1-q}\, /\, r_{q}"
)

for (label, prof) in profs_ana
    x = LinRange(-2, 2, 1000)
    rh = LilGuys.calc_R_h(prof)

    y = log10.(LilGuys.calc_Σ.(prof, rh * 10 .^ x))
    y .-= log10(LilGuys.calc_Σ(prof, rh))
    
    lines!(x, y, label = label)
end

ylims!(-5, 1.5)
axislegend()

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "quantile",
    ylabel = L"\log\,r_{1-q}\, /\, r_{q}"
)

y_exp = values(r_quant_exp_2d) |> collect
for (label, rqs) in r_quants
    x = keys(rqs) |> collect
    y = values(rqs) |> collect
    scatterlines!(x, log10.(y), label = label)
end

axislegend()

fig

In [ ]:
galaxies = [
    "aquarius2",
    "bootes1",
    "bootes2",
    "bootes3",
    "bootes4",
    "bootes5",
    "canesvenatici1",
    "canesvenatici2",
    "carina",
    "carina2",
    "carina3",
    "centaurus1",
    "cetus2",
    "columba1",
    "coma_berenices",
    "crater2",
    # "delve2", needs properties
    #DES 2 stars
    "draco",
    "draco2",
    "eridanus2",
    # "eridanus3", needs Mv
    "eridanus4",
    "fornax",
    # "grus1", fits err
    "grus2",
    #"hercules",
    "horologium1",
    "horologium2",
    "hydra2",
    "hydrus1",
    #"indus1", needs Mv
    #"indus2", needs Mv, no stars
    "leo1",
    "leo2",
    "leo4",
    "leo5",
     "leo_minor",
     "leo_t",
    # "pegasus3", no stars
     "pegasus4", 
    # "phoenix", fits err
     "phoenix2",
    "pictor2",
   # "pictoris1", obs props
    # "pisces2", 2 stars
    "reticulum2",
    "reticulum3",
    #"sagittarius2", needs Mv
    "sculptor",
    "segue1",
    "segue2",
    "sextans1",
    "triangulum2",
    "tucana2",
    "tucana3",
    "tucana4",
    "tucana5",
    # "unions1", needs Mv
    "ursa_major1",
    "ursa_major2",
    "ursa_minor",
    "virgo1",
    # "virgo2" 2 stars,
    "willman1",
]

In [ ]:
galaxies_flagged = [
    "bootes1",
    "bootes3",
    "draco2",
    "grus2",
    "sculptor",
    "segue1",
    "tucana2",
    "tucana3",
    "ursa_minor",
    ]

In [ ]:
profiles_flagged = OrderedDict(key => load_profile(key, ext="_2c") for key in galaxies_flagged)

In [ ]:
profiles = OrderedDict(key => load_profile(key) for key in galaxies)

In [ ]:
fig = Figure(
    size = (400, 600)
    
)
ax = Axis(fig[1,1], xlabel = "log r / kpc", ylabel=L"\log\,\Sigma")
i = 0

dy = 1.0
for label in galaxies_flagged
    color=COLORS[(i) % 7+1]
    
    prof = profiles[label]
    
    y = prof.log_Sigma .+ i*dy
    errscatter!(prof.log_r, y, yerr=prof.log_Sigma_err, color=color)
    lines!(prof.log_r, y)

    prof = profiles_flagged[label]
    
    y = prof.log_Sigma .+ i*dy
    lines!(prof.log_r, y, color=:black, linestyle=:dash)
    
    text!(-2.5, 0+i*dy, text=label, color=color)
    i += 1
end

fig

In [ ]:
for galaxy in galaxies_flagged
    profiles[galaxy] = profiles_flagged[galaxy]
end

In [ ]:
profiles["bootes1"].log_r_quantiles

In [ ]:
properties = OrderedDict(key => load_props(key) for key in galaxies)

## Sigma

In [ ]:
p = profiles["sculptor"]

In [ ]:
using Measurements

In [ ]:
f = LilGuys.lerp(p.log_r .± 0, p.log_Sigma .± p.log_Sigma_err)

In [ ]:
function Σ_ioo(profile::LilGuys.StellarProfile; l=-1.5, h=0.5)
    f = LilGuys.lerp(profile.log_r .± 0, profile.log_Sigma .± profile.log_Sigma_err)
    if h > profile.log_r[end]
        println("warning, skipping")
        return NaN±NaN
    end
    if l < profile.log_r[1]
        println("warning, skipping")
        return NaN±NaN
    end  

    println(f(h), "\t", f(l))

    return exp10(f(h)) / exp10(f(l))
end

In [ ]:
function Σ_ioo(prof::LilGuys.SphericalProfile; l=-1.5, h=0.5)
    f = x -> LilGuys.calc_Σ(prof, 10 .^ x)
    return f(h) / f(l)
end

In [ ]:
function plot_Σ_iio(l, h)
    
    for galaxy in galaxies
        println(galaxy)
        s = Σ_ioo(profiles[galaxy], l=l, h=h)
        properties[galaxy]["Sigma_ioo"] = Measurements.value.(s)
        properties[galaxy]["Sigma_ioo_err"] = Measurements.uncertainty.(s)
        
    end
    
    fig = Figure()
    
    ax = Axis(fig[1,1], 
        xlabel = "Mv",
        ylabel = L"\log\ \Sigma_{%$h}/\Sigma_{%$l}",
        limits = (-15, 1, nothing, nothing),
        xreversed = true
    )
    hlines!(log10(Σ_ioo(profs_ana["exp3d"], l=l, h=h)), color=:black, label="exp3d")
    
    for (i, galaxy) in enumerate(galaxies)
        if galaxy ∈ galaxies_flagged
            markersize=15
        else
            markersize=8
        end
        color = COLORS[(i+1) % length(COLORS) + 1]
        
        x = properties[galaxy]["Mv"]
        xerr = properties[galaxy]["Mv_err"]

        y = log10.(properties[galaxy]["Sigma_ioo"])
        yerr = properties[galaxy]["Sigma_ioo_err"] / properties[galaxy]["Sigma_ioo"] / log(10)

        errscatter!([x], [y], yerr = [yerr], xerr=[xerr], color=color, size=markersize)
        text!(x, y, text=galaxy, fontsize=12, color=color)
    end
    
    fig
end

In [ ]:
plot_Σ_iio(-2.0, 0.5)

In [ ]:
plot_Σ_iio(-1.5, 0.5)

In [ ]:
plot_Σ_iio(-1.0, 0.5)

In [ ]:
plot_Σ_iio(-0.5, 0.5)

In [ ]:
plot_Σ_iio(-0.3, 0.5)

In [ ]:
plot_Σ_iio(-0.5, 0.3)

In [ ]:
plot_Σ_iio(-0.3, 0.3)

In [ ]:
plot_Σ_iio(-0.5, 0.7)

In [ ]:
plot_Σ_iio(-0.5, 1.0)

## Quantiles

In [ ]:


function plot_r_ooi(q)
    
    for galaxy in galaxies
        properties[galaxy]["r_ioo"] = r_ioo(profiles[galaxy], q=q)
    end
    
    fig = Figure()
    
    ax = Axis(fig[1,1], 
        xlabel = "Mv",
        ylabel = L"\log\ r_{%$(1-q)}/r_{%$q}",
        limits = (-15, 1, nothing, nothing),
        xreversed = true
    )
    hlines!(log10(r_ioo(LilGuys.Exp3D(), q=q)), color=:black, label="exp3d")
    hlines!(log10(r_ioo(LilGuys.Plummer(), q=q)), color=:black, linestyle=:dot, label="plummer")
    
    for (i, galaxy) in enumerate(galaxies)
        if galaxy ∈ galaxies_flagged
            markersize=20
        else
            markersize=10
        end
        color = COLORS[(i+1) % length(COLORS) + 1]
        
        x = properties[galaxy]["Mv"]
        y = log10.(properties[galaxy]["r_ioo"])
        scatter!(x, y, markersize=markersize, color=color)
        text!(x, y, text=galaxy, fontsize=12, color=color)
    
    
    end
    
    fig
end

In [ ]:
plot_r_ooi(0.25)

In [ ]:
plot_r_ooi(0.16)

In [ ]:
plot_r_ooi(0.05)

In [ ]:
plot_r_ooi(0.01)

# R Out / median

In [ ]:
function plot_room(q)
    
    fig = Figure()
    
    ax = Axis(fig[1,1], 
        xlabel = "Mv",
        ylabel = L"r_{%$(1-q)}/r_h",
        limits = (-15, 0.5, nothing, nothing),
        xreversed = true
    )
    hlines!(log10(r_oom(LilGuys.Exp3D(), q=q)), color=:black, label="exp3d")
    hlines!(log10(r_oom(LilGuys.Plummer(), q=q)), color=:black, linestyle=:dot, label="plummer")
    
    for (i, galaxy) in enumerate(galaxies)
        if galaxy ∈ galaxies_flagged
            markersize=20
        else
            markersize=10
        end
        color = COLORS[(i+1) % length(COLORS) + 1]
        
        x = properties[galaxy]["Mv"]
        y = log10.(r_oom(profiles[galaxy], q=q))
        scatter!(x, y, markersize=markersize, color=color)
        text!(x, y, text=galaxy, fontsize=12, color=color)
    
    
    end
    
    fig
end

In [ ]:
plot_room(0.02)

In [ ]:
plot_room(0.05)

In [ ]:
plot_room(0.16)

In [ ]:
plot_room(0.25)

In [ ]:
for (label, prof) in profiles
    println(label)
    rh = LilGuys.quantile(prof.log_r, prof.counts, 0.5)

    println(rh, "\t", 10^rh)
    println("counts = ", sum(prof.counts))
    println("Mv = ", properties[label]["Mv"])
    println("log rhme / rh = ", profiles[label].log_r_quantiles[profiles[label].quantiles .== 0.5])

    println()
end

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "log r / r_h",
    ylabel=L"\log\,\Sigma",
   # limits=(-2, 2, -5, 5)
)

i = 0
colorrange=(-13, 0)
colormap = Reverse(Arya.get_arya_cmap())

for (label, prof) in profiles
    y = prof.log_Sigma

    if length(y) < 1
        continue
    end
    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=colorrange, colormap=colormap)
end


Colorbar(fig[1,2], colorrange=colorrange, label="Mv (mag)", colormap=colormap)
fig

In [ ]:
prof = LilGuys.Exp3D(r_s = 1/2.026996389650345, M=1)

In [ ]:
fig = Figure(size=(700, 400))
ax = Axis(fig[1,1], 
    xlabel = L"\log\, r\, /\, r_h", 
    ylabel=L"\log\,\Sigma\,/\,\Sigma_h"
)
i = 0

for (label, prof) in profiles
    y = prof.log_Sigma
    if  length(y) <= 2
        continue
    else
        print(label, ", ")
    end

    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=colorrange, colormap=colormap)
end

x = LinRange(-2, 0.75, 1000)
y = log10.(LilGuys.calc_Σ.(prof_ana, exp10.(x))) .- log10(LilGuys.calc_Σ(prof, 1))
lines!(x, y, color=COLORS[1])

Colorbar(fig[1,2], colorrange=colorrange, label=L"$M_V$ (mag)", colormap=colormap)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "log r / r_h", 
    ylabel=L"\log\,\Sigma",
    title="dwarfs with features",
)
i = 0

for (label, prof) in profiles
    y = prof.log_Sigma
    if label ∉ galaxies_flagged
        continue
    end

    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=colorrange, colormap=colormap)
end


Colorbar(fig[1,2], colorrange=colorrange, label="Mv (mag)", colormap=colormap)

x = LinRange(-2, 0.75, 1000)
y = log10.(LilGuys.calc_Σ.(prof_ana, exp10.(x))) .- log10(LilGuys.calc_Σ(prof, 1))
lines!(x, y, color=COLORS[1])



fig

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], 
    xlabel = "log r / r_h", 
    ylabel=L"\log\,\Sigma",
    title="dwarfs without",
)
i = 0

for (label, prof) in profiles
    y = prof.log_Sigma
    if label ∈ galaxies_flagged || (sum(prof.counts) < 30)
        continue
    end

    color = properties[label]["Mv"]
    
    scatterlines!(prof.log_r, y, color=color, colorrange=(-10, 0))
end

x = LinRange(-2, 0.75, 1000)
y = log10.(LilGuys.calc_Σ.(prof, exp10.(x))) .- log10(LilGuys.calc_Σ(prof, 1))
lines!(x, y, color=COLORS[1])


fig

In [ ]:
fig = Figure(
    size = (700, 1200),
    
)
ax = Axis(fig[1,1], 
    xlabel = "log r / kpc", 
    ylabel=L"\log\,\Sigma",
    xticks=[-2, -1, 0, 1, 2],
)
i = 0

dy = 1.0
for (label, prof) in profiles
    y = prof.log_Sigma .+ i*dy
    if length(y) < 2
        continue
    end
    
    errscatter!(prof.log_r, y, yerr=prof.log_Sigma_err)
    lines!(prof.log_r, y)

    text!(-3, i*dy, text=label, color=COLORS[(i) % 7+1])
    i += 1
end

fig

In [ ]:
function scatter_obs(df)
    fig = Figure()
    ax = Axis(fig[1,1],
        aspect=DataAspect())

    scatter!(df.xi, df.eta, markersize=1, alpha=0.1, color=:black)
    scatter!(df.xi[df.PSAT .> 0.2], df.eta[df.PSAT .> 0.2], markersize=3, color=COLORS[2])

    LilGuys.hide_grid!(ax)
    fig
end

In [ ]:
df = load_gaia_obs("bootes1")

# Galaxy specific

TODO, want to move this to a pluto notebook

In [ ]:
function load_gaia_obs(galaxyname)
    df = LilGuys.read_fits(joinpath(galaxyname, "data", "j24_2c.fits"))
    return df
end

In [ ]:
scatter_obs(df)

In [ ]:
df = load_gaia_obs("segue1")

In [ ]:
scatter_obs(df)

In [ ]:
df = load_gaia_obs("draco2")

In [ ]:
scatter_obs(df)

In [ ]:
df = load_gaia_obs("grus2")

In [ ]:
scatter_obs(df)